In [1]:
from experiment_runner import *

In [2]:
self = ExperimentRunner(cuda=True, seq_len=20, model_name = 'bert', method=105)

In [3]:
self.prepare_data()
self.prepare_model()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [282]:
NUMBER_OF_ROWS = 5
f_row = list(self.test.iterrows())[0:NUMBER_OF_ROWS]

In [30]:
row = [x[1] for x in f_row]
# row

In [32]:
row_sent = [x[1]['sentence'] for x in f_row]


In [31]:
# row[2]['sentence']

'Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second_Europan War who perform_ secret black _operations and are pitted_against the Imperial unit " Calamaty Raven " .'

In [35]:
encoded_row =  self.tokenizer(row_sent, padding=True,  truncation=True, max_length=self.SEQ_LEN, return_tensors ='pt').input_ids
if self.CUDA:
    encoded_row = encoded_row.cuda()
print(encoded_row)
    

tensor([[  101, 11748,  4801,  4360,  1997,  1996, 11686,  1017,  1007,  1010,
          4141,  3615,  2000,  2004, 11748,  4801,  4360, 11906,  3523,   102],
        [  101,  2207,  1999,  2254,  2249,  1999,  2900,  1010,  2009,  2003,
          1996,  2353,  2208,  1999,  1996, 11748,  4801,  4360,  1035,   102],
        [  101, 15440,  1996,  2168, 10077,  1997,  8608,  1998,  2613,  1030,
          1011,  1030,  2051, 11247,  2004,  2049, 16372,  1010,  1996,   102],
        [  101,  1996,  2208,  2211,  2458,  1999,  2230,  1010,  4755,  1035,
          2058,  1037,  2312,  4664,  1997,  1996,  2147,  1035,  2589,   102],
        [  101,  2096,  2009,  6025,  1996,  3115,  2838,  1997,  1996,  2186,
          1010,  2009,  2036,  9601,  3674, 24081,  1010,  2107,  1035,   102]],
       device='cuda:0')


In [275]:
import itertools

NUMBER_OF_PERM = 10
### L and R starts from 0
l = 1
r = 3
perms = generate_perm(NUMBER_OF_PERM)




def generate_perm(nperm):
    perm_stored=[]
    arg = np.arange(self.SEQ_LEN)
    i = 0
    while i < nperm: 
        perm_stored.append(np.random.permutation(arg))
        perm_stored.append(perm_stored[-1][::-1])
        i += 1
    return np.array(perm_stored)


def generate_incremental_masks(permutations, l, r):
    n_rows, n_cols = permutations.shape
    
    masks = np.zeros((n_rows, n_cols), dtype=np.int8)
    
    stop_mask = (permutations == l) | (permutations == r)
    stop_indices = np.argmax(stop_mask, axis=1)
    row_indices = np.arange(n_rows)
    col_indices = np.arange(n_cols)[np.newaxis, :]
    valid_mask = col_indices < stop_indices[:, np.newaxis]
    indices = permutations * valid_mask

    
    for inde, row in enumerate(indices):
        non_zero_indices = row[row != 0]
        masks[inde, non_zero_indices] = True
    return masks

In [ ]:
def get_smax(encoded_row, masks_tensor):
    expanded_input = encoded_row.unsqueeze(0).expand(masks_tensor.size(0), -1, -1)
    masks_tensor_expanded = masks_tensor.unsqueeze(1).expand(-1, encoded_row.shape[0], -1)
    masked_tensors = torch.where(masks_tensor_expanded == 1, expanded_input, torch.tensor(self.tokenizer.pad_token_id, device='cuda:0'))

    masked_tensors = masked_tensors.flatten(start_dim = 0, end_dim = 1)
    # masked_tensors = masked_tensors.reshape((masked_tensors.shape[0] * masked_tensors.shape[1], -1))

    
    masks_tensor_expanded = masks_tensor_expanded.flatten(start_dim = 0, end_dim = 1)
    # masks_tensor_expanded = masks_tensor_expanded.reshape((masks_tensor_expanded.shape[0] * masks_tensor_expanded.shape[1], -1))
    logits = self.model(masked_tensors, attention_mask=masks_tensor_expanded).logits    
    smax = logits.softmax(dim=-1)
    return smax, logits
    

masks = generate_incremental_masks(perms, l, r)
masks_tensor = torch.tensor(masks, dtype=torch.long, device='cuda:0')

smax_ab, logits_ab = get_smax(encoded_row, masks_tensor.clone())

masks_tensor_A = masks_tensor.clone()
masks_tensor_A[:, l] = 1
smax_a, logits_a = get_smax(encoded_row, masks_tensor_A)

masks_tensor_B = masks_tensor.clone()
masks_tensor_B[:, r] = 1
smax_b, logits_b = get_smax(encoded_row, masks_tensor_B)

masks_tensor_phi = masks_tensor.clone()
masks_tensor_phi[:, r] = 1
masks_tensor_phi[:, l] = 1
smax_phi, logits_phi = get_smax(encoded_row, masks_tensor_phi)


smax = smax_ab - smax_a - smax_b + smax_phi
logits = logits_ab - logits_a - logits_b + logits_phi


smax = smax.unflatten(0, (smax.shape[0]//encoded_row.shape[0], encoded_row.shape[0]))
logits = logits.unflatten(0, (logits.shape[0]//encoded_row.shape[0], encoded_row.shape[0]))
print(smax.shape, logits.shape)

In [338]:
torch.linalg.norm(smax,dim=-1).mean(axis=0).shape

torch.Size([5, 20])

In [91]:
# Apply all masks at once

masked_tensors

tensor([[[    0,     0,     0,  ...,     0,     0,   102],
         [    0,     0,     0,  ...,     0,     0,   102],
         [    0,     0,     0,  ...,     0,     0,   102],
         [    0,     0,     0,  ...,     0,     0,   102],
         [    0,     0,     0,  ...,     0,     0,   102]],

        [[    0,     0,     0,  ...,     0,     0,   102],
         [    0,     0,     0,  ...,     0,     0,   102],
         [    0,     0,     0,  ...,     0,     0,   102],
         [    0,     0,     0,  ...,     0,     0,   102],
         [    0,     0,     0,  ...,     0,     0,   102]],

        [[    0,     0,     0,  ...,     0,     0,   102],
         [    0,     0,     0,  ...,     0,     0,   102],
         [    0,     0,     0,  ...,     0,     0,   102],
         [    0,     0,     0,  ...,     0,     0,   102],
         [    0,     0,     0,  ...,     0,     0,   102]],

        ...,

        [[    0, 11748,  4801,  ..., 11906,  3523,   102],
         [    0,  2207,  1999,  ...,

In [92]:
self.tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [97]:
self.tokenizer.sep_token_id

102

In [ ]:
row = list(self.test.iterrows())[0][1]

In [57]:
encoded_row =  self.tokenizer(row['sentence'], padding=False,  truncation=True, max_length=self.SEQ_LEN, return_tensors ='pt').input_ids.cuda().cuda()

In [58]:
self.model(encoded_row)

MaskedLMOutput(loss=None, logits=tensor([[[ -8.5790,  -8.7265,  -8.6121,  ...,  -7.8080,  -7.6920,  -4.1057],
         [ -8.9574,  -8.9489,  -8.9837,  ...,  -8.1245,  -8.0452,  -5.1740],
         [ -6.6033,  -6.7179,  -6.5225,  ...,  -4.8117,  -5.6250,  -2.1881],
         ...,
         [-11.6282, -12.2846, -11.9921,  ..., -11.4965, -12.0546,  -8.5730],
         [-12.9916, -13.0614, -13.1172,  ..., -11.6188, -12.3721,  -7.7752],
         [-14.7684, -14.7193, -14.6344,  ..., -14.1674, -12.2202, -10.8975]]],
       device='cuda:0', grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [ ]:
import shap

In [35]:
encoded_row

tensor([[    0,     0,  4801,     0,  1997,     0, 11686,  1017,     0,  1010,
          4141,     0,  2000,  2004, 11748,     0,  4360,     0,     0,   102]])

In [62]:
def masker(binary_mask, X):
    X[binary_mask] = self.tokenizer.pad_token_id
    X = X.reshape(1, -1)
    return X

print(type(self.model))
shap.PermutationExplainer(self.model, masker, seed=42)(encoded_row)

<class 'transformers.models.bert.modeling_bert.BertForMaskedLM'>


TypeError: 'int' object is not callable

In [ ]:
# X_train =  self.tokenizer(self.test['sentence'].to_list(), padding=True,  truncation=True, max_length=self.SEQ_LEN, return_tensors ='pt').input_ids
# X_train =  self.tokenizer(self.test['sentence'].to_list(), padding=True,  truncation=True, max_length=self.SEQ_LEN, return_tensors ='np').input_ids


In [ ]:
X_train.shape

In [ ]:
    # vfunc = nshap.vfunc.interventional_shap(lambda x : self.model(x).logits, X_train, target=None, num_samples=1000)

In [ ]:
encoded_row =  self.tokenizer(row['sentence'], padding=False,  truncation=True, max_length=self.SEQ_LEN, return_tensors ='pt').input_ids

self.model(encoded_row.reshape(1, -1))

In [ ]:
encoded_row.reshape(1, -1).shape

In [ ]:
encoded_row =  self.tokenizer(row['sentence'], padding=False,  truncation=True, max_length=self.SEQ_LEN, return_tensors ='np').input_ids

print(encoded_row.shape)
# if self.CUDA:
#     encoded_row = encoded_row.cuda()
if len(row['weak_mwe']) != 0:
    vfunc = nshap.vfunc.interventional_shap(lambda x : self.model(x).logits, X_train, target=None, num_samples=1000)
    # vfunc = interventional_shap(lambda x : self.model(x).logits, X_train, target=None, num_samples=1000)
    shapley_taylor = nshap.shapley_taylor(encoded_row, vfunc, n=2)
    print(shapley_taylor)

In [ ]:
self.CUDA